# Item 39. 태그 클래스보다는 클래스 계층을 사용하라

- Enum을 사용해 클래스의 상태를 정의하는 방식을 태그 클래스라 표현
- 태그 클래스는 여러 상태에 대해 하나의 클래스에서 제어해야하기 때문에 책임 분리 측면에서 좋지 않음


In [2]:
class ValueMatcher<T> private constructor(
    private val value: T,
    private val matcher: Matcher
) {
    enum class Matcher {
        EQUALS,
        CONTAINS,
        STARTS_WITH,
        LIST_EMPTY
    }
    
    fun matches(other: T): Boolean {
        return when (matcher) {
            Matcher.EQUALS -> value == other
            Matcher.CONTAINS -> other.toString().contains(value.toString())
            Matcher.STARTS_WITH -> other.toString().startsWith(value.toString())
            Matcher.LIST_EMPTY -> (other as List<*>).isEmpty()
        }
    }
    
    companion object {
        fun <T> equals(value: T) = ValueMatcher(value, Matcher.EQUALS)
        fun <T> contains(value: T) = ValueMatcher(value, Matcher.CONTAINS)
        fun <T> startsWith(value: T) = ValueMatcher(value, Matcher.STARTS_WITH)
        fun <T> listEmpty() = ValueMatcher(null, Matcher.LIST_EMPTY)
    }
}


Line_2.jupyter.kts (3:33 - 36) No type arguments expected for enum class Matcher
Line_2.jupyter.kts (13:13 - 27) Comparison of incompatible enums '[Error type: Type for error type constructor (Matcher)]<T>' and 'Line_2_jupyter.ValueMatcher.Matcher' is always unsuccessful
Line_2.jupyter.kts (14:13 - 29) Comparison of incompatible enums '[Error type: Type for error type constructor (Matcher)]<T>' and 'Line_2_jupyter.ValueMatcher.Matcher' is always unsuccessful
Line_2.jupyter.kts (15:13 - 32) Comparison of incompatible enums '[Error type: Type for error type constructor (Matcher)]<T>' and 'Line_2_jupyter.ValueMatcher.Matcher' is always unsuccessful

- 위처럼 ValueMatcher 클래스가 모든 상태에 대해 알고 핸들링해야한다.
- 여러 목적을 가지고 요소를 여러방법으로 설정할 수 있는경우에는 상태의 일관성과 정확성을 지기키 어렵다
- 물론 설계를 위처럼 하지 않고 더 나은 설계로 매쳐를 enum 클래스를 싱글턴 디자인 패턴처럼 사용해서 사용할 수 도 있음
- 하지만 코틀린에서는 보다 나은 sealed class를 사용할 수 있다!

## sealed class

- 태그 클래스보다 sealed class를 사용하면 상태를 나타내는 클래스 계층을 손쉽게 만들 수 있다.



In [ ]:
sealed class Matcher<T> {
    abstract fun matches(value: T): Boolean
    
    class Equals<T>(private val value: T) : Matcher<T>() {
        override fun matches(other: T) = value == other
    }
    
    class Contains<T>(private val value: T) : Matcher<T>() {
        override fun matches(other: T) = other.toString().contains(value.toString())
    }
    
    class StartsWith<T>(private val value: T) : Matcher<T>() {
        override fun matches(other: T) = other.toString().startsWith(value.toString())
    }
    
    class ListEmpty<T> : Matcher<List<*>>() {
        override fun matches(other: List<*>) = other.isEmpty()
    }
}



## sealed class 장점

- 반드시 같은 파일 내에서만 상속을 허용하므로, 상태를 제한해야하는 상황에서 더 유용하다. 반대로 확장에 닫혀있는편
- 컴파일 레벨에서 모든 분기를 확인하게되므로 모든 상태를 다루는지 확인할 수 있다.(when 절 사용시 매우 편리)
- 자연스럽게 태그클래스방식대신 상태 패턴으로 설계를 유도하여 보다 나은 구조가 된다 

Hello!
Hello!
